In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append('../')

from utils.cleaning import int2rom
from utils.sample_data import load_odyssey
from nltk.corpus import wordnet, words
import nltk
import re
import json
import numpy as np
import pandas as pd
import itertools as its
from  itertools import permutations as PNR

import matplotlib.pyplot as plt

# Markov chains

How does graph connectivity change as a function of probability threshold?

Relationship between rank of a word and its connectivity?

In [3]:
text_words,text_sents=load_odyssey()
start_tokens, end_tokens = set(), set()
for s in text_sents:
    w = nltk.word_tokenize(s)
    start_tokens.add(w[0])
    end_tokens.add(w[len(w)-1])

In [4]:
def transition_mat(text):
    '''Where text is word_tokenized version of text'''
    unique = set(text)
    un_ind = {} #indicies for each unique word
    for uw in unique:
        un_ind[uw] = [i for i, x in enumerate(text) if x == uw] #for some reason having indices is faster?

    trans = pd.DataFrame(0, columns = unique, index = unique)
    for (w1,w2) in PNR(unique,2): #avoid double loop
        for i in un_ind[w1]:
            try: 
                if text[i+1] == w2: trans.loc[w1][w2] += 1
            except: pass
#         for i, w in enumerate(text):
#             if w == w1 and text[i+1] == w2:
#                 trans.loc[w1][w2] += 1
    return trans.div(trans.sum(axis=1),axis=0)       

In [5]:
trans = transition_mat(text_words)

In [ ]:
plt.hist(
    np.array([trans[col].unique() for col in trans.columns]).flatten())

In [9]:
# from https://www.analyticsvidhya.com/blog/2021/04/generating-fantasy-titles-with-a-markov-chain/
def markov_sentence(trans, start_tokens, end_tokens,thresh=0):
    trans = trans.copy(deep=True) #if a copy isn't made it will permanently change the df
    trans[trans < thresh] = 0
    start_tokens = list(start_tokens)
    
    sent = ''
    word = start_tokens[np.random.randint(len(start_tokens))]
    cols = list(trans.columns)
    col_i = 0
    rand = np.random.rand()
    while word not in end_tokens:
        if col_i >= len(cols):
            print(col_i)
        w2 = cols[col_i]
        p = trans.loc[word,w2]
        if rand <= p:
            sent += word + ' '
            word = w2
            col_i = 0
            rand = np.random.rand()
        else:
            rand -= p
            col_i += 1
    return sent

In [16]:
markov_sentence(trans,start_tokens,end_tokens,thresh=0.001)

'demodocus s gear tumbled right hands of a tremendous sea with which case some god '

In [ ]:
x = pd.DataFrame(1, columns = ['a','b','c'], index= ['e','f','g'])
x.loc['e']['b'] =2
x=x.div(x.sum(axis=1),axis=0)
def do(y):
    y[y>.25] = 3
do(x)
x